[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arvidl/ELMED219-2026/blob/main/Lab3-GenAI-LLM/notebooks/04-prompt-engineering.ipynb)

# Prompt Engineering for Helsefaglige Oppgaver

**ELMED219 / BMED365 - Lab 3**

---

## Læringsmål

Etter denne notebooken skal du kunne:
- Mestre grunnleggende prompt-teknikker
- Anvende zero-shot og few-shot learning
- Implementere chain-of-thought for komplekse vurderinger
- Designe trygge prompts for medisinske oppgaver

## Innhold

1. [Grunnleggende prinsipper](#1-grunnleggende-prinsipper)
2. [Zero-shot vs. Few-shot](#2-zero-shot-vs-few-shot)
3. [Chain-of-Thought (CoT)](#3-chain-of-thought-cot)
4. [Strukturert output](#4-strukturert-output)
5. [Sikkerhet i medisinske prompts](#5-sikkerhet-i-medisinske-prompts)
6. [Praktiske eksempler](#6-praktiske-eksempler)

---

## 1. Grunnleggende prinsipper

God prompt engineering handler om å kommunisere tydelig med AI-modellen. Tenk på det som å gi instruksjoner til en svært kunnskapsrik, men bokstavelig assistent.

### De fire grunnprinsippene

| Prinsipp | Beskrivelse | Eksempel |
|----------|-------------|----------|
| **Spesifisitet** | Vær konkret om hva du vil ha | "Skriv 3 punkter" vs. "Skriv litt" |
| **Kontekst** | Gi relevant bakgrunnsinformasjon | "Du er en allmennlege som..." |
| **Format** | Spesifiser ønsket output-format | "Svar i JSON-format" |
| **Eksempler** | Vis hva du forventer | Few-shot learning |

### Eksempel: Dårlig vs. god prompt

**Dårlig prompt:**
```
Fortell meg om diabetes
```

**God prompt:**
```
Du er en erfaren allmennlege som forklarer diagnoser til pasienter.

Forklar diabetes type 2 til en nydiagnostisert pasient på 55 år.

Inkluder:
1. Hva som skjer i kroppen (2-3 setninger, enkelt språk)
2. Vanlige symptomer (punktliste, maks 5)
3. Hvorfor behandling er viktig (2 setninger)
4. Tre konkrete livsstilsråd de kan starte med i dag

Bruk vennlig, beroligende tone. Unngå medisinsk sjargong.
Maks 150 ord totalt.
```

In [1]:
# La oss analysere forskjellene

darlig_prompt = "Fortell meg om diabetes"

god_prompt = """Du er en erfaren allmennlege som forklarer diagnoser til pasienter.

Forklar diabetes type 2 til en nydiagnostisert pasient på 55 år.

Inkluder:
1. Hva som skjer i kroppen (2-3 setninger, enkelt språk)
2. Vanlige symptomer (punktliste, maks 5)
3. Hvorfor behandling er viktig (2 setninger)
4. Tre konkrete livsstilsråd de kan starte med i dag

Bruk vennlig, beroligende tone. Unngå medisinsk sjargong.
Maks 150 ord totalt."""

print("DÅRLIG PROMPT:")
print(f"  Lengde: {len(darlig_prompt)} tegn")
print(f"  Mangler: Rolle, målgruppe, format, begrensninger")
print()
print("GOD PROMPT:")
print(f"  Lengde: {len(god_prompt)} tegn")
print(f"  Inneholder:")
print(f"    - Rolle: 'erfaren allmennlege'")
print(f"    - Målgruppe: 'nydiagnostisert pasient på 55 år'")
print(f"    - Struktur: 4 spesifikke punkter")
print(f"    - Tone: 'vennlig, beroligende'")
print(f"    - Begrensning: 'Maks 150 ord'")

DÅRLIG PROMPT:
  Lengde: 23 tegn
  Mangler: Rolle, målgruppe, format, begrensninger

GOD PROMPT:
  Lengde: 422 tegn
  Inneholder:
    - Rolle: 'erfaren allmennlege'
    - Målgruppe: 'nydiagnostisert pasient på 55 år'
    - Struktur: 4 spesifikke punkter
    - Tone: 'vennlig, beroligende'
    - Begrensning: 'Maks 150 ord'


---

## 2. Zero-shot vs. Few-shot

### Zero-shot
Modellen løser oppgaven uten eksempler - bare basert på instruksjoner.

### Few-shot
Vi gir modellen noen eksempler på ønsket input-output før den faktiske oppgaven.

### Når bruke hva?

| Situasjon | Anbefalt teknikk |
|-----------|------------------|
| Enkel, kjent oppgave | Zero-shot |
| Spesifikt format/stil | Few-shot |
| Domenespesifikk terminologi | Few-shot |
| Klassifisering med fasit | Few-shot |

In [2]:
# Eksempel: Triage-klassifisering

# ZERO-SHOT: Ingen eksempler
zero_shot_prompt = """Klassifiser følgende symptombeskrivelse som enten 'AKUTT' eller 'IKKE-AKUTT':

"Pasienten rapporterer brystsmerter som stråler til venstre arm, 
kortpustethet og kvalme de siste 30 minuttene."

Svar kun med klassifiseringen."""

print("ZERO-SHOT PROMPT:")
print("-" * 50)
print(zero_shot_prompt)
print()

# FEW-SHOT: Med eksempler
few_shot_prompt = """Klassifiser symptombeskrivelser som 'AKUTT' eller 'IKKE-AKUTT'.

Eksempler:

Symptomer: "Hodepine i 3 dager, forverres gradvis"
Klassifisering: IKKE-AKUTT

Symptomer: "Plutselig kraftig hodepine, stiv nakke, feber"
Klassifisering: AKUTT

Symptomer: "Lett forkjølelse med rennende nese"
Klassifisering: IKKE-AKUTT

Symptomer: "Plutselig synstap på ett øye for 15 minutter siden"
Klassifisering: AKUTT

Nå klassifiser:
Symptomer: "Pasienten rapporterer brystsmerter som stråler til venstre arm, 
kortpustethet og kvalme de siste 30 minuttene."
Klassifisering:"""

print("FEW-SHOT PROMPT:")
print("-" * 50)
print(few_shot_prompt)

ZERO-SHOT PROMPT:
--------------------------------------------------
Klassifiser følgende symptombeskrivelse som enten 'AKUTT' eller 'IKKE-AKUTT':

"Pasienten rapporterer brystsmerter som stråler til venstre arm, 
kortpustethet og kvalme de siste 30 minuttene."

Svar kun med klassifiseringen.

FEW-SHOT PROMPT:
--------------------------------------------------
Klassifiser symptombeskrivelser som 'AKUTT' eller 'IKKE-AKUTT'.

Eksempler:

Symptomer: "Hodepine i 3 dager, forverres gradvis"
Klassifisering: IKKE-AKUTT

Symptomer: "Plutselig kraftig hodepine, stiv nakke, feber"
Klassifisering: AKUTT

Symptomer: "Lett forkjølelse med rennende nese"
Klassifisering: IKKE-AKUTT

Symptomer: "Plutselig synstap på ett øye for 15 minutter siden"
Klassifisering: AKUTT

Nå klassifiser:
Symptomer: "Pasienten rapporterer brystsmerter som stråler til venstre arm, 
kortpustethet og kvalme de siste 30 minuttene."
Klassifisering:


### Fordeler med Few-shot

1. **Konsistens**: Modellen følger mønsteret fra eksemplene
2. **Nøyaktighet**: Reduserer misforståelser
3. **Format**: Sikrer riktig output-format
4. **Domenetilpasning**: Viser ønsket terminologi og stil

---

## 3. Chain-of-Thought (CoT)

Chain-of-Thought ber modellen vise sin tankeprosess steg for steg. Dette er spesielt nyttig for:

- Komplekse resonnementer
- Diagnostiske vurderinger
- Matematiske beregninger
- Beslutninger med flere faktorer

### Hvorfor fungerer det?

Når modellen "tenker høyt", må den strukturere resonnementet logisk, noe som ofte fører til bedre svar.

In [3]:
# Eksempel: Differensialdiagnose med og uten CoT

case = """65 år gammel mann med diabetes type 2 og hypertensjon.
Innkommer med akutte brystsmerter, svetting og kvalme.
BT 160/100, puls 95, temperatur normal."""

# UTEN Chain-of-Thought
simple_prompt = f"""Case: {case}

Spørsmål: Bør pasienten legges inn?
Svar kort."""

print("UTEN Chain-of-Thought:")
print("=" * 50)
print(simple_prompt)
print()

# MED Chain-of-Thought
cot_prompt = f"""Case: {case}

Vurder denne casen steg-for-steg:

STEG 1: Identifiser risikofaktorer
- List opp pasientens risikofaktorer for hjertesykdom

STEG 2: Vurder symptomenes alvorlighetsgrad
- Er symptomene typiske for et bestemt tilstandsbilde?
- Hva er differensialdiagnosene?

STEG 3: Vurder vitale tegn
- Er blodtrykk og puls bekymringsfulle?

STEG 4: Konklusjon
- Basert på steg 1-3, bør pasienten legges inn?
- Begrunn svaret.

Vis din tankeprosess for hvert steg."""

print("MED Chain-of-Thought:")
print("=" * 50)
print(cot_prompt)

UTEN Chain-of-Thought:
Case: 65 år gammel mann med diabetes type 2 og hypertensjon.
Innkommer med akutte brystsmerter, svetting og kvalme.
BT 160/100, puls 95, temperatur normal.

Spørsmål: Bør pasienten legges inn?
Svar kort.

MED Chain-of-Thought:
Case: 65 år gammel mann med diabetes type 2 og hypertensjon.
Innkommer med akutte brystsmerter, svetting og kvalme.
BT 160/100, puls 95, temperatur normal.

Vurder denne casen steg-for-steg:

STEG 1: Identifiser risikofaktorer
- List opp pasientens risikofaktorer for hjertesykdom

STEG 2: Vurder symptomenes alvorlighetsgrad
- Er symptomene typiske for et bestemt tilstandsbilde?
- Hva er differensialdiagnosene?

STEG 3: Vurder vitale tegn
- Er blodtrykk og puls bekymringsfulle?

STEG 4: Konklusjon
- Basert på steg 1-3, bør pasienten legges inn?
- Begrunn svaret.

Vis din tankeprosess for hvert steg.


### Tips for effektiv CoT

1. **Eksplisitte steg**: Nummer stegene tydelig
2. **Spesifikke spørsmål**: Still konkrete spørsmål for hvert steg
3. **Logisk rekkefølge**: Bygg opp mot konklusjonen
4. **Be om begrunnelse**: "Forklar hvorfor" er kraftfullt

---

## 4. Strukturert output

For programmatisk bruk er det nyttig å be om strukturert output som JSON.

In [4]:
# Eksempel: Strukturert journalnotat

structured_prompt = """Analyser følgende konsultasjonsnotater og generer et strukturert sammendrag.

KONSULTASJON:
Pasient 45 år, kvinne. Kommer pga tretthet siste 2 måneder.
Sover dårlig, våkner flere ganger. Stresset på jobb.
Spiser lite frokost, mye kaffe. Ingen mosjon.
Tidligere frisk. Tar ingen medisiner.
Undersøkelse: BT 125/80, puls 78. Upåfallende.
Blodprøver: Hb 11.2, ferritin 18 (lav-normal).
Vurderer jernmangel og søvnproblemer som hovedårsaker.

Generer et SOAP-notat i følgende JSON-format:

{
  "subjektivt": {
    "hovedklage": "...",
    "varighet": "...",
    "assosierte_symptomer": ["..."],
    "livsstil": "..."
  },
  "objektivt": {
    "vitale": "...",
    "undersokelse": "...",
    "laboratorium": "..."
  },
  "vurdering": {
    "hoveddiagnose": "...",
    "differensialdiagnoser": ["..."]
  },
  "plan": [
    "..."
  ]
}"""

print(structured_prompt)

Analyser følgende konsultasjonsnotater og generer et strukturert sammendrag.

KONSULTASJON:
Pasient 45 år, kvinne. Kommer pga tretthet siste 2 måneder.
Sover dårlig, våkner flere ganger. Stresset på jobb.
Spiser lite frokost, mye kaffe. Ingen mosjon.
Tidligere frisk. Tar ingen medisiner.
Undersøkelse: BT 125/80, puls 78. Upåfallende.
Blodprøver: Hb 11.2, ferritin 18 (lav-normal).
Vurderer jernmangel og søvnproblemer som hovedårsaker.

Generer et SOAP-notat i følgende JSON-format:

{
  "subjektivt": {
    "hovedklage": "...",
    "varighet": "...",
    "assosierte_symptomer": ["..."],
    "livsstil": "..."
  },
  "objektivt": {
    "vitale": "...",
    "undersokelse": "...",
    "laboratorium": "..."
  },
  "vurdering": {
    "hoveddiagnose": "...",
    "differensialdiagnoser": ["..."]
  },
  "plan": [
    "..."
  ]
}


In [5]:
# Simulert respons for demonstrasjon
import json

simulert_respons = {
    "subjektivt": {
        "hovedklage": "Tretthet",
        "varighet": "2 måneder",
        "assosierte_symptomer": ["Søvnvansker", "Hyppig oppvåkning"],
        "livsstil": "Høyt stressnivå, uregelmessige måltider, mye koffein, ingen mosjon"
    },
    "objektivt": {
        "vitale": "BT 125/80, puls 78",
        "undersokelse": "Upåfallende",
        "laboratorium": "Hb 11.2 g/dL, Ferritin 18 ng/mL (lav-normal)"
    },
    "vurdering": {
        "hoveddiagnose": "Jernmangel med begynnende anemi",
        "differensialdiagnoser": ["Søvnforstyrrelse", "Stressrelatert utmattelse", "Hypotyreose"]
    },
    "plan": [
        "Jerntilskudd 100mg daglig",
        "Søvnhygiene-råd",
        "Kontroll TSH for å utelukke stoffskiftesykdom",
        "Kontroll om 6-8 uker med nye blodprøver"
    ]
}

print("STRUKTURERT OUTPUT (JSON):")
print("=" * 50)
print(json.dumps(simulert_respons, indent=2, ensure_ascii=False))

STRUKTURERT OUTPUT (JSON):
{
  "subjektivt": {
    "hovedklage": "Tretthet",
    "varighet": "2 måneder",
    "assosierte_symptomer": [
      "Søvnvansker",
      "Hyppig oppvåkning"
    ],
    "livsstil": "Høyt stressnivå, uregelmessige måltider, mye koffein, ingen mosjon"
  },
  "objektivt": {
    "vitale": "BT 125/80, puls 78",
    "undersokelse": "Upåfallende",
    "laboratorium": "Hb 11.2 g/dL, Ferritin 18 ng/mL (lav-normal)"
  },
  "vurdering": {
    "hoveddiagnose": "Jernmangel med begynnende anemi",
    "differensialdiagnoser": [
      "Søvnforstyrrelse",
      "Stressrelatert utmattelse",
      "Hypotyreose"
    ]
  },
  "plan": [
    "Jerntilskudd 100mg daglig",
    "Søvnhygiene-råd",
    "Kontroll TSH for å utelukke stoffskiftesykdom",
    "Kontroll om 6-8 uker med nye blodprøver"
  ]
}


---

## 5. Sikkerhet i medisinske prompts

Når vi bruker LLM-er for medisinske oppgaver, må vi inkludere sikkerhetsinstruksjoner.

### Viktige elementer

1. **Ansvarsfraskrivelse**: AI erstatter ikke klinisk vurdering
2. **Henvisning**: Anbefal alltid å kontakte helsepersonell
3. **Begrensninger**: Vær tydelig på hva AI ikke kan/bør gjøre
4. **Usikkerhet**: Be modellen uttrykke usikkerhet når relevant

In [6]:
# Mal for sikker medisinsk prompt

sikker_prompt_mal = """ROLLE:
Du er en medisinsk informasjonsassistent som gir generell helseinformasjon.

VIKTIGE BEGRENSNINGER:
1. Du gir IKKE medisinske diagnoser eller behandlingsanbefalinger
2. Du erstatter IKKE konsultasjon med kvalifisert helsepersonell
3. Ved akutte symptomer: Anbefal alltid å kontakte lege/113
4. Vær tydelig på usikkerhet og begrensninger i din kunnskap
5. Aldri gi spesifikke medisindoseringer

OPPGAVE:
[Din oppgave her]

FORMAT:
Start svaret med: "Denne informasjonen er kun veiledende og erstatter ikke medisinsk rådgivning."
Avslutt med: "Kontakt lege for personlig vurdering."
"""

print("MAL FOR SIKKER MEDISINSK PROMPT:")
print("=" * 50)
print(sikker_prompt_mal)

MAL FOR SIKKER MEDISINSK PROMPT:
ROLLE:
Du er en medisinsk informasjonsassistent som gir generell helseinformasjon.

VIKTIGE BEGRENSNINGER:
1. Du gir IKKE medisinske diagnoser eller behandlingsanbefalinger
2. Du erstatter IKKE konsultasjon med kvalifisert helsepersonell
3. Ved akutte symptomer: Anbefal alltid å kontakte lege/113
4. Vær tydelig på usikkerhet og begrensninger i din kunnskap
5. Aldri gi spesifikke medisindoseringer

OPPGAVE:
[Din oppgave her]

FORMAT:
Start svaret med: "Denne informasjonen er kun veiledende og erstatter ikke medisinsk rådgivning."
Avslutt med: "Kontakt lege for personlig vurdering."



In [7]:
# Eksempel: Sikker håndtering av symptomspørsmål

usikker_sporsmaal = "Hvor mye Paracetamol kan jeg ta for sterke smerter?"

sikker_prompt = f"""ROLLE:
Du er en medisinsk informasjonsassistent.

VIKTIGE BEGRENSNINGER:
1. Aldri gi spesifikke doseringsanbefalinger
2. Henvis alltid til pakningsvedlegg eller apotek
3. Ved usikkerhet, anbefal å kontakte lege

SPØRSMÅL FRA BRUKER:
{usikker_sporsmaal}

SVAR PÅ EN ANSVARLIG MÅTE:"""

print("SPØRSMÅL:", usikker_sporsmaal)
print()
print("SIKKER PROMPT:")
print("-" * 50)
print(sikker_prompt)
print()
print("FORVENTET TYPE SVAR:")
print("-" * 50)
print("""Jeg kan ikke gi spesifikke doseringsanbefalinger da dette avhenger av 
flere faktorer som din alder, vekt, andre medisiner og helsetilstand.

Generelt anbefaler jeg:
1. Les pakningsvedlegget nøye
2. Spør på apoteket for veiledning
3. Ved sterke eller vedvarende smerter, kontakt lege

Ved overdosering eller bivirkninger, kontakt Giftinformasjonen (22 59 13 00).""")

SPØRSMÅL: Hvor mye Paracetamol kan jeg ta for sterke smerter?

SIKKER PROMPT:
--------------------------------------------------
ROLLE:
Du er en medisinsk informasjonsassistent.

VIKTIGE BEGRENSNINGER:
1. Aldri gi spesifikke doseringsanbefalinger
2. Henvis alltid til pakningsvedlegg eller apotek
3. Ved usikkerhet, anbefal å kontakte lege

SPØRSMÅL FRA BRUKER:
Hvor mye Paracetamol kan jeg ta for sterke smerter?

SVAR PÅ EN ANSVARLIG MÅTE:

FORVENTET TYPE SVAR:
--------------------------------------------------
Jeg kan ikke gi spesifikke doseringsanbefalinger da dette avhenger av 
flere faktorer som din alder, vekt, andre medisiner og helsetilstand.

Generelt anbefaler jeg:
1. Les pakningsvedlegget nøye
2. Spør på apoteket for veiledning
3. Ved sterke eller vedvarende smerter, kontakt lege

Ved overdosering eller bivirkninger, kontakt Giftinformasjonen (22 59 13 00).


---

## 6. Praktiske eksempler

Her er noen ferdiglagde prompts for vanlige medisinske oppgaver.

In [8]:
# EKSEMPEL 1: Journalsammendrag

journalsammendrag_prompt = """OPPGAVE: Lag et kortfattet sammendrag av følgende pasientjournal.

JOURNAL:
15.01.2024: Diabetes kontroll. HbA1c 7.2%, vekt 89 kg. Fortsetter metformin.
03.02.2024: Telefon - spørsmål om ny medisin. Forklarte bivirkninger.
20.02.2024: Akutt time - UVI. Dysuri, pollakisuri. Urin-stix pos. Pivmecillinam 5 dager.
15.03.2024: Diabetes kontroll. HbA1c 6.9%, ned 0.3%. Vekt 87 kg. Veldig fornøyd.

FORMAT:
Skriv et sammendrag på maks 3 setninger som dekker:
- Hoveddiagnose og utvikling
- Viktigste hendelser
- Nåværende status"""

print("EKSEMPEL 1: Journalsammendrag")
print("=" * 50)
print(journalsammendrag_prompt)

EKSEMPEL 1: Journalsammendrag
OPPGAVE: Lag et kortfattet sammendrag av følgende pasientjournal.

JOURNAL:
15.01.2024: Diabetes kontroll. HbA1c 7.2%, vekt 89 kg. Fortsetter metformin.
03.02.2024: Telefon - spørsmål om ny medisin. Forklarte bivirkninger.
20.02.2024: Akutt time - UVI. Dysuri, pollakisuri. Urin-stix pos. Pivmecillinam 5 dager.
15.03.2024: Diabetes kontroll. HbA1c 6.9%, ned 0.3%. Vekt 87 kg. Veldig fornøyd.

FORMAT:
Skriv et sammendrag på maks 3 setninger som dekker:
- Hoveddiagnose og utvikling
- Viktigste hendelser
- Nåværende status


In [9]:
# EKSEMPEL 2: Pasientvennlig forklaring

pasientforklaring_prompt = """ROLLE: Du er en allmennlege som forklarer medisinske tilstander til pasienter.

OPPGAVE: Forklar følgende tilstand på en forståelig måte.

TILSTAND: Atrieflimmer
PASIENT: 68 år, kvinne, bekymret

KRAV:
1. Bruk enkelt språk - unngå medisinsk sjargong
2. Forklar hva som skjer i hjertet (bruk gjerne en analogi)
3. Nevn vanlige symptomer
4. Forklar hvorfor behandling er viktig
5. Vær beroligende men ærlig
6. Maks 100 ord

EKSEMPEL PÅ GOD ANALOGI:
"Tenk på hjertet som en dirigent som holder takten. Ved atrieflimmer 
mister dirigenten litt av rytmen, så orkesteret spiller litt ujevnt."""

print("EKSEMPEL 2: Pasientvennlig forklaring")
print("=" * 50)
print(pasientforklaring_prompt)

EKSEMPEL 2: Pasientvennlig forklaring
ROLLE: Du er en allmennlege som forklarer medisinske tilstander til pasienter.

OPPGAVE: Forklar følgende tilstand på en forståelig måte.

TILSTAND: Atrieflimmer
PASIENT: 68 år, kvinne, bekymret

KRAV:
1. Bruk enkelt språk - unngå medisinsk sjargong
2. Forklar hva som skjer i hjertet (bruk gjerne en analogi)
3. Nevn vanlige symptomer
4. Forklar hvorfor behandling er viktig
5. Vær beroligende men ærlig
6. Maks 100 ord

EKSEMPEL PÅ GOD ANALOGI:
"Tenk på hjertet som en dirigent som holder takten. Ved atrieflimmer 
mister dirigenten litt av rytmen, så orkesteret spiller litt ujevnt.


In [10]:
# EKSEMPEL 3: Triage-vurdering

triage_prompt = """OPPGAVE: Vurder hastegrad for følgende symptombeskrivelse.

BRUK FØLGENDE KATEGORIER:
- RØD: Livstruende, krever umiddelbar hjelp (ring 113)
- ORANSJE: Haster, bør vurderes av lege innen timer
- GUL: Bør vurderes av lege innen 1-2 dager  
- GRØNN: Kan vente, egnetfor fastlege/egenbehandling

SYMPTOMBESKRIVELSE:
[Sett inn symptomer her]

SVAR FORMAT:
1. Hastegrad: [RØD/ORANSJE/GUL/GRØNN]
2. Begrunnelse: [2-3 setninger]
3. Anbefalt handling: [Konkret råd]
4. Røde flagg å følge med på: [Liste]

VIKTIG: Dette er kun veiledende. Ved usikkerhet, velg høyere hastegrad."""

print("EKSEMPEL 3: Triage-vurdering")
print("=" * 50)
print(triage_prompt)

EKSEMPEL 3: Triage-vurdering
OPPGAVE: Vurder hastegrad for følgende symptombeskrivelse.

BRUK FØLGENDE KATEGORIER:
- RØD: Livstruende, krever umiddelbar hjelp (ring 113)
- ORANSJE: Haster, bør vurderes av lege innen timer
- GUL: Bør vurderes av lege innen 1-2 dager  
- GRØNN: Kan vente, egnetfor fastlege/egenbehandling

SYMPTOMBESKRIVELSE:
[Sett inn symptomer her]

SVAR FORMAT:
1. Hastegrad: [RØD/ORANSJE/GUL/GRØNN]
2. Begrunnelse: [2-3 setninger]
3. Anbefalt handling: [Konkret råd]
4. Røde flagg å følge med på: [Liste]

VIKTIG: Dette er kun veiledende. Ved usikkerhet, velg høyere hastegrad.


---

## Oppsummering

### Beste praksis for medisinske prompts

**DO:**
- Spesifiser rolle og kontekst
- Gi eksempler (few-shot) for konsistens
- Bruk stegvis resonnering (CoT) for komplekse vurderinger
- Inkluder alltid sikkerhetsinstruksjoner
- Be om strukturert output når relevant

**DON'T:**
- Stol blindt på AI for kritiske beslutninger
- Be om spesifikke medisindoseringer
- Glem å inkludere ansvarsfraskrivelser
- Ignorer usikkerhet i AI-svar

### Refleksjonsoppgaver

1. Design en prompt for å generere pasientinformasjon om en valgfri tilstand
2. Hvordan ville du modifisert triage-prompten for å håndtere barn spesifikt?
3. Hvilke sikkerhetshensyn er viktigst i din fremtidige praksis?

---

*Neste notebook: [05 - Forklarbar AI (XAI)](05-xai-forklarbar-ai.ipynb)*